In [1]:
import numpy as np
import matplotlib.pyplot as plt

from src.input import Input_uc, Input_ed
from src.output import Output_uc, Output_ed
from src.unit_commitment import solve_uc
from src.economic_dispatch import solve_ed

In [2]:
num_units=3
num_periods=6
num_buses=1

COPYPASTE = np.zeros(num_periods)

In [24]:
input_uc = Input_uc(
    # meta
    num_units=num_units,
    num_periods=num_periods,
    num_buses=num_buses,
    voll=3500*1300,
    let_blackout=False,
    curtail_penalty=0,
    let_curtail=False,
    exact_reserve=False,
    # renewable
    solar_p_max=COPYPASTE.copy(),
    solar_p_min=COPYPASTE.copy(),
    wind_p=COPYPASTE.copy(),
    hydro_p=COPYPASTE.copy(),
    # system
    load=[220, 250, 200, 170, 230, 190],
    system_reserve_up=[10, 10, 10, 10, 10, 10],
    system_reserve_down=COPYPASTE.copy(), # [10, 10, 10, 0, 10, 10], 
    # operational constraint
    p_min=[80, 50, 30],
    p_max=[300, 200, 100],
    ramp_up=[50, 60, 70],
    ramp_down=[30, 40, 50],
    startup_ramp=[100, 70, 40],
    shutdown_ramp=[80, 50, 30],
    min_up=[3, 2, 1],
    min_down=[2, 2, 2],
    # generation cost function
    cost_quad=[0, 0, 0],
    cost_lin=[5, 15, 30],
    cost_const=[0, 0, 0],
    # previous horizon
    p_prev=[120, 0, 0],
    u_prev=[
        [1],
        [0],
        [0],
    ],
    # startup cost function
    cost_startup_step=[
        [800],
        [500],
        [250],
    ],
    mustoff=None,
)

print(f"min_down_prev computed internally from u_prev input: {input_uc.min_down_prev}")
print(f"this is different from the toy example question min_down_prev {[0, 0, 0]}")
print(f"because number of cooling steps for each i (bar tau i) = {input_uc.num_cooling_steps}")
print(f"which is the 'shape' of nested list u_prev structure")
print(f"min_down = {input_uc.min_down} which is the least number of u_prev we need")
print(f"min_up was identical to toy example problem setup\n")
print(f"THIS IS NOT A PROBLEM IN THE ACTUAL PROJECT USING KPG BECAUSE")
print(f"bar tau i is equal or greater than min_up or min_down")
input_uc.min_down_prev = [0, 0, 0]

min_down_prev computed internally from u_prev input: [0, 1, 1]
this is different from the toy example question min_down_prev [0, 0, 0]
because number of cooling steps for each i (bar tau i) = [1, 1, 1]
which is the 'shape' of nested list u_prev structure
min_down = [2, 2, 2] which is the least number of u_prev we need
min_up was identical to toy example problem setup

THIS IS NOT A PROBLEM IN THE ACTUAL PROJECT USING KPG BECAUSE
bar tau i is equal or greater than min_up or min_down


In [5]:
output_uc = Output_uc()
solve_uc(input_uc=input_uc, output_uc=output_uc)

Set parameter Username
Set parameter LicenseID to value 2659802
Academic license - for non-commercial use only - expires 2026-05-01


In [6]:
output_ed = Output_ed(num_periods=num_periods, num_units=num_units, num_buses=num_buses)

for time_period in range(num_periods):
    input_ed = Input_ed(
        # meta
        time_period=time_period,
        num_units=num_units,
        num_buses=num_buses,
        voll=3500*1300,
        let_blackout=False,
        curtail_penalty=0,
        let_curtail=False,
        exact_reserve=False,
        # renewable
        solar_p_max=COPYPASTE.copy(),
        solar_p_min=COPYPASTE.copy(),
        # uc
        input_uc=input_uc,
        output_uc=output_uc,
    )

    solve_ed(input_ed=input_ed, output_ed=output_ed)

output_ed.compute_auxiliary_results()
output_uc.compute_auxiliary_results(output_ed=output_ed)

In [7]:
output_uc.__dict__

{'total_cost_system': 8800.0,
 'total_cost_generation': 8050.0,
 'total_cost_startup': 750.0,
 'total_cost_voll': 0.0,
 'total_cost_curtail_penalty': 0.0,
 'total_cost_reserve_up': 0.0,
 'total_cost_reserve_down': 0.0,
 'cost_system': array([2100., 1750., 1000.,  850., 2150.,  950.]),
 'cost_generation': array([1600., 1750., 1000.,  850., 1900.,  950.]),
 'cost_startup': array([500.,   0.,   0.,   0., 250.,   0.]),
 'cost_voll': array([0., 0., 0., 0., 0., 0.]),
 'cost_curtail_penalty': array([0., 0., 0., 0., 0., 0.]),
 'cost_reserve_up': array([0., 0., 0., 0., 0., 0.]),
 'cost_reserve_down': array([0., 0., 0., 0., 0., 0.]),
 'u': array([[ 1.,  1.,  1.,  1.,  1.,  1.],
        [ 1.,  1., -0., -0., -0., -0.],
        [-0., -0., -0., -0.,  1., -0.]]),
 'z': array([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.]]),
 'p': array([[170., 200., 200., 170., 200., 190.],
        [ 50.,  50.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.,  30.,   0.]]),
 'r_u